# TensorFlow

## グラフの作成とセッション内での実行

In [2]:
import tensorflow as tf

In [3]:
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

このコードは計算グラフを作るだけで、実際変数でさえ初期化されていない。このグラフを評価するには、`TensorFlow セッション`を開き、それを使用して変数を初期化し、fを評価する。

In [4]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
result

42

In [5]:
sess.close()

In [6]:
# 別のやり方1
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
result

42

In [7]:
# 別のやり方2(Jupyterはこれを推奨)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()
result

42

`TensolFlow`は一般的に二つの部分に分割される。　一つ目が計算グラフを作る`構築フェーズ`、二つ目がそれを実行する`実行フェーズ`。

## グラフの管理
作成したノートは、自動的にデフォルトのグラフに追加される。

In [8]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

複数の独立したグラフを管理したい場合は、withブロックでそれを一時的にデフォルトグラフにする

In [9]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

In [10]:
x2.graph is graph

True

In [11]:
x2.graph is tf.get_default_graph()

False

## ノード値のライフサイクル

In [12]:
w = tf.constant(3)

x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


## TensorFlowによる線形回帰

In [13]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [14]:
housing = fetch_california_housing()
m, n = housing.data.shape

In [15]:
m, n

(20640, 8)

In [16]:
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data] # バイアス項1を付与
housing_data_plus_bias[:2]

array([[ 1.00000000e+00,  8.32520000e+00,  4.10000000e+01,
         6.98412698e+00,  1.02380952e+00,  3.22000000e+02,
         2.55555556e+00,  3.78800000e+01, -1.22230000e+02],
       [ 1.00000000e+00,  8.30140000e+00,  2.10000000e+01,
         6.23813708e+00,  9.71880492e-01,  2.40100000e+03,
         2.10984183e+00,  3.78600000e+01, -1.22220000e+02]])

In [17]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

In [18]:
XT = tf.transpose(X)

In [19]:
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

In [20]:
with tf.Session() as sess:
    theta_value = theta.eval()
theta_value

array([[-3.6894890e+01],
       [ 4.3661433e-01],
       [ 9.4453208e-03],
       [-1.0704148e-01],
       [ 6.4345831e-01],
       [-3.9632569e-06],
       [-3.7880042e-03],
       [-4.2093179e-01],
       [-4.3400639e-01]], dtype=float32)

NumPyで正規方程式を計算するのに比べTensorFlowはGPUが使える場合はGPUでこれを実行してくれる

## 勾配下降法の実装
正規方程式ではなく、バッチ勾配下降法を使用してみる。

### マニュアルの勾配計算
まず、手動で勾配を計算する

In [21]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

勾配下降法を使用するときには、まず入力特徴量行列を正規化することが大切だ。正規化を行わないと訓練が大幅に遅くなる。  
scikit-learnのStandardScalerを使用して正規化を行なった

In [22]:
n_epochs = 1000
learning_rate = 0.01

In [23]:
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')

In [24]:
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name='theta') # thetaをランダムな値に設定

In [25]:
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')# 二乗誤差を計算

tf.assign()は変数に新しい値を代入するノードを作成する。  
この場合は、バッチ勾配下降法のステップ$\theta^{(next step)} = \theta - \eta\nabla_\theta MSE(\theta)$を実装している

In [26]:
gradients = 2/m * tf.matmul(tf.transpose(X), error) # 勾配を計算
training_op = tf.assign(theta, theta - learning_rate * gradients) 

In [27]:
init = tf.global_variables_initializer()

In [28]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE =', mse.eval())
            sess.run(training_op)

Epoch 0 MSE = 11.570529
Epoch 100 MSE = 11.121117
Epoch 200 MSE = 10.691737
Epoch 300 MSE = 10.281421
Epoch 400 MSE = 9.889257
Epoch 500 MSE = 9.51438
Epoch 600 MSE = 9.155969
Epoch 700 MSE = 8.813242
Epoch 800 MSE = 8.48546
Epoch 900 MSE = 8.171931


### 自動微分を使った方法
TensorFlowの自動微分機能を使用して微分を行う

In [29]:
gradientsa = tf.gradients(mse, [theta])[0]

gradients()は入力としてオペレーション(この場合はmse)と変数リスト(この場合はtheta)を与えるとops(変数ごとに一つずつ)のリストを作成し、個々の変数に対応する勾配を計算する。

### オプティマイザを使った方法
TensorFlowは、勾配下降オプティマイザを含む様々なオプティマイザを提供している。

In [30]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_opain = optimizer.minimize(mse)

## 訓練アルゴリズムへのデータの供給
ミニバッチ勾配下降法のコードを置き換える。もっとも簡単な方法はプレースホルダノードを使うものである。プレースホルダノードは実際には計算を行わず、実行時に出力せよと指示したデータを出力する特殊なノードである。

In [31]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5

with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})

In [32]:
B_val_1

array([[6., 7., 8.]], dtype=float32)

In [33]:
B_val_2

array([[ 9., 10., 11.],
       [12., 13., 14.]], dtype=float32)

In [34]:
# ミニバッチ勾配下降法を書き換える
housing = fetch_california_housing()
m, n = housing.data.shape

n_epochs = 10

X = tf.placeholder(tf.float32, shape=(None, n+1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')

In [35]:
batch_size = 100
n_batches = int(np.ceil(m/batch_size))

In [36]:
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [37]:
def fetch_batch(epoch, batch_index, batch_size):
    indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

In [38]:
best_theta

array([[ 2.078745  ],
       [ 0.8243525 ],
       [ 0.14286363],
       [-0.24867727],
       [ 0.3100895 ],
       [-0.00332945],
       [-0.01753671],
       [-0.9070387 ],
       [-0.8822807 ]], dtype=float32)

## モデルの保存と復元

In [39]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions") 
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
            # 100エポックごとにチェックポイントを設定
            save_path = saver.save(sess, "models/my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "models/my_model_final.ckpt")

Epoch 0 MSE = 2.754427
Epoch 100 MSE = 0.63222194
Epoch 200 MSE = 0.5727803
Epoch 300 MSE = 0.5585009
Epoch 400 MSE = 0.54907006
Epoch 500 MSE = 0.542288
Epoch 600 MSE = 0.5373791
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.53124255
Epoch 900 MSE = 0.5293704


In [41]:
with tf.Session() as sess:
    saver.restore(sess, "models/my_model.ckpt")
    best_theta_restored = theta.eval() # not shown in the book

In [47]:
best_theta_restored

array([[-0.10788631],
       [ 0.39412713],
       [ 0.41932344],
       [-0.03573537],
       [-0.73307323],
       [-0.9637971 ],
       [ 0.33473706],
       [ 0.5449357 ],
       [ 0.64875317]], dtype=float32)

In [49]:
saver = tf.train.Saver({"weights": theta})

In [53]:
saver = tf.train.import_meta_graph("models/my_model_final.ckpt.meta")
theta = tf.get_default_graph().get_tensor_by_name("theta:0")

with tf.Session() as sess:
    saver.restore(sess, "models/my_model_final.ckpt")
    best_theta_restored = theta.eval()

In [55]:
best_theta_restored

array([[-0.10788631],
       [ 0.39412713],
       [ 0.41932344],
       [-0.03573537],
       [-0.73307323],
       [-0.9637971 ],
       [ 0.33473706],
       [ 0.5449357 ],
       [ 0.64875317]], dtype=float32)

## TonsorFlowBoardを使用したグラフの訓練曲線の可視化

In [58]:
from datetime import datetime

In [ ]:
now = datetimea